In [9]:
import sys
sys.path.append('..')

from pseq_helpers import *
import pypulseq as pp
import datetime
import pickle

In [2]:
# Initialize the sequence base
pseq0 = Pseq_Base(channels = ("x", "y", "z"), max_grad=35, max_slew=150)

# Plays skope trigger
pseq_excite = PSeq_Skope_Trigger()

# Both blip and chirp test waveforms
pseq_test_blip = PSeq_WaveTest_Blip(pseq0, do_adc = False)
pseq_test_chirp = PSeq_WaveTest_Chirp(pseq0, do_adc = False)

# Refocuses moments (not strictly necessary)
pe_refocus_areas = None
pseq_refocus_blip = PSeq_Refocus_PE(pseq0, areas0=pe_refocus_areas, areas1=pe_refocus_areas, areas2=pseq_test_blip.all_refocus_areas)
pseq_refocus_chirp = PSeq_Refocus_PE(pseq0, areas0=pe_refocus_areas, areas1=pe_refocus_areas, areas2=pseq_test_chirp.all_refocus_areas)

In [3]:
TR = 500e-3
N_av = 10

pseq0.init_seq()
all_meas = []

for i_av in range(N_av):
    for channels in [("y", "z", "x"), ("x", "z", "y"), ("x", "y", "z")]:
        pseq0.channels = channels
        for mode in ['blip', 'chirp']:
            if mode == 'blip':
                pseq_test_wave, pseq_refocus = [pseq_test_blip, pseq_refocus_blip]
            elif mode == 'chirp':
                pseq_test_wave, pseq_refocus = [pseq_test_chirp, pseq_refocus_chirp]
            for idx in range(12):  # Different blips/chirps for test waveforms
                for polarity in [-1, 0, 1]:  # The 0 polarity can be removed here for better efficiency
                    
                    pseq0.add_pseq_to_self(pseq_excite)
                    pseq0.add_pseq_to_self(pseq_test_wave, idx, polarity)
                    pseq0.add_pseq_to_self(pseq_refocus, ie2 = idx, polarity2 = polarity)
                    
                    dur = (pseq_excite.get_duration() 
                    + pseq_test_wave.get_duration(idx, polarity) 
                    + pseq_refocus.get_duration(ie2 = idx, polarity2 = polarity)
                    )

                    pseq0.add_delay(TR-dur)
                    
                    all_meas.append(
                        {'channels': channels,
                        'mode': mode,
                        'idx': idx,
                        'polarity': polarity,
                        'i_av': i_av,
                        }
                    )
                
pseq0.add_dummy_adc()

In [6]:
seq_time = pseq0.get_seq_time()
print('Seq Time:', datetime.timedelta(seconds=round(seq_time)))

# Error "p_vit = ..." is from calculating trajectory during empty ADC, seems fine
ok, error_report = pseq0.seq.check_timing()
if ok:
    print("Timing check passed successfully")
else:
    print("Timing check failed. Error listing follows:")
    [print(e) for e in error_report]

Seq Time: 0:18:00
Timing check passed successfully


In [ ]:
wave_data, tfp_excitation, tfp_refocusing, t_adc, fp_adc = pseq0.seq.waveforms_and_times(append_RF=True)
t_trigger = get_trigger_times(pseq0.seq)

# This contains all of the information needed to process the GIRF (and then some)
save_res = {'all_meas': all_meas, 
            'wave_data':wave_data,
            'tfp_excitation':tfp_excitation,
            'tfp_refocusing':tfp_refocusing,
            't_adc':t_adc,
            'fp_adc':fp_adc,
            't_trigger': t_trigger,
            }


In [ ]:
seq_name = f'skope_meas'
print(f'{seq_name = }')
pseq0.seq.set_definition(key="FOV", value=200e-3)
pseq0.seq.set_definition(key="Name", value=seq_name)

pseq0.seq.write(f'./seqs/{seq_name}.seq')
with open(f'./seqs/{seq_name}.pickle', 'wb') as handle:
    pickle.dump(save_res, handle, protocol=pickle.HIGHEST_PROTOCOL)

seq_name = 'skope_meas'
